In [1]:
import random
import numpy as np


def score(context, response):
    return random.random()


def best_response(context, candidates):
    index = np.argmax([score(context, response) for response in candidates])
    return candidates[index]


def parse_dialogs(filename):
    dialogs = []
    with open(filename, 'r') as f:
        dialog = []
        for line in f:
            if line.strip() == '':
                dialogs.append(dialog)
                dialog = []
            else:
                user_utt, bot_utt = line.strip().split('\t')
                utt_num = user_utt.split(' ')[0]
                user_utt = ' '.join(user_utt.split(' ')[1:])
                dialog.append((utt_num, user_utt, bot_utt))
    return dialogs


def parse_candidates(filename):
    with open(filename, 'r') as f:
        return [' '.join(line.strip().split(' ')[1:]) for line in f]            

    
def responses_accuracy(dialogs, candidates):
    correct = 0
    count = 0
    for dialog in dialogs:
        for _, user_utt, bot_utt in dialog:
            count += 1
            context = user_utt
            response = best_response(context, candidates)
            if response == bot_utt:
                correct += 1
    return correct / count, correct, count

In [2]:
dev_set_task1_dialogs = parse_dialogs('dataset/dialog-bAbI-tasks/dialog-babi-task1-API-calls-dev.txt')
candidates = parse_candidates('dataset/dialog-bAbI-tasks/dialog-babi-candidates.txt')

In [3]:
responses_accuracy(dev_set_task1_dialogs, candidates)

(0.00016625103906899418, 1, 6015)

`f(x,y) = (Ax)^T*By`, `A, B <- {DxV}, x <- V, y <- V`

1. Надо инциализировать все переменные

In [21]:
V = 100
D = 20

x = np.random.choice([0, 1], size=(V,1), p=(9/10, 1/10))
y = np.random.choice([0, 1], size=(V,1), p=(9/10, 1/10))
y_neg = np.random.choice([0, 1], size=(V,1), p=(9/10, 1/10))
A = np.random.randn(D, V)

x.shape, y.shape, A.shape

((100, 1), (100, 1), (20, 100))

In [5]:
import tensorflow as tf

In [14]:
context = tf.placeholder(dtype=tf.float32, name='Context', shape=[V, 1])
neg_context = tf.placeholder(dtype=tf.float32, name='NegativeContext', shape=[V, 1])
response = tf.placeholder(dtype=tf.float32, name='Response', shape=[V, 1])
A_var = tf.Variable(tf.random_uniform([D, V], -1.0, 1.0))
B_var = tf.Variable(tf.random_uniform([D, V], -1.0, 1.0))

In [15]:
f_pos = tf.matmul(tf.transpose(tf.matmul(A_var, context)), tf.matmul(B_var, response))
f_neg = tf.matmul(tf.transpose(tf.matmul(A_var, neg_context)), tf.matmul(B_var, response))

In [16]:
m = 0.01
loss = f_pos - f_neg + m

In [17]:
LR = 0.01
optimizer = tf.train.GradientDescentOptimizer(LR).minimize(loss)

In [29]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    avg_loss = 0.0
    for ind in range(1000):
        loss_elem = sess.run([loss, optimizer], feed_dict={context: y, response: x, neg_context: y_neg})
        avg_loss += loss_elem[0][0]
        if ind % 100 == 0 :
            print(loss_elem[0][0], avg_loss / ind)

/usr/local/var/pyenv/versions/3.6.0/lib/python3.6/site-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/var/pyenv/versions/3.6.0/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: overflow encountered in add


[ 15.25640297] [ inf]
[ -5.13647638e+13] [ -2.12560747e+12]
[ -5.29116354e+25] [ -1.09481037e+24]
[ -5.45050714e+37] [ -7.51853950e+35]
[-inf] [-inf]
[-inf] [-inf]
[-inf] [-inf]
[ nan] [ nan]
[ nan] [ nan]
[ nan] [ nan]


In [8]:
def ranking_margin_objective(output, margin=1.0):
    ''' Create final model output and loss for pairwise ranking margin objective
        
    Loss for single pair (f(p), f(n)) = [margin - f(p) + f(n)]+
    This only works when given model output on alternating positive/negative 
    pairs: [pos,neg,pos,neg,...]. TODO: check target placeholder
    at runtime to make sure this is the case?
        
    Args:
        output: Model output
        margin: The margin value for the pairwise hinge loss
    Returns:
        tuple (final output, loss)
    '''
    y_pairs = tf.reshape(output, [-1,2]) # fold: 1 x n -> [n/2 x 2]
    pos_scores, neg_scores = tf.split(1, 2, y_pairs) # separate pairs
    hinge_losses = tf.nn.relu(margin - pos_scores + neg_scores)
    total_hinge_loss = tf.reduce_sum(hinge_losses)
    return output, total_hinge_loss